<a href="https://colab.research.google.com/github/DeepaliVerma/personal_files/blob/main/Custom_layer_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist

In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 28*28).astype("float32")/255.0
x_test = x_test.reshape(-1, 28*28).astype("float32")/255.0

11501568/11490434 [==============================] - 0s 0us/step


In [4]:
#class without input tensor shape need build function
#method1
class Dense(layers.Layer):
  def __init__(self, units):
    super(Dense, self).__init__()
    self.units = units


  def call(self, inputs):
    return tf.matmul(inputs, self.w) + self.b

  def build(self, input_shape):
    self.w = self.add_weight(
        name='w',
        shape=(input_shape[-1], self.units),
        initializer='random_normal',
        trainable=True,
    )

    self.b = self.add_weight(name='b', shape=(self.units,), initializer='zeros', trainable=True,)


In [5]:
#Method2
#class with input tensor
class Dense(layers.Layer):
  def __init__(self, units, input_dim):
    super(Dense, self).__init__()
    self.w = self.add_weight(
        name='w',
        shape=(input_dim, units),
        initializer='random_normal',
        trainable=True,
    )

    self.b = self.add_weight(name='b', shape=(units,), initializer='zeros', trainable=True,)

  def call(self, inputs):
    return tf.matmul(inputs, self.w) + self.b



In [10]:
#customize relu
class MyReLU(layers.Layer):
  def __init__(self):
    super(MyReLU, self).__init__()

  def call(self, x):
    return tf.math.maximum(x,0)
    

In [11]:
#model
class MyModel(keras.Model):
  def __init__(self, num_classes=10):
    super(MyModel, self).__init__()
    #new dense layer function
    # self.dense1 = Dense(64, 784)
    # self.dense2 = Dense(10,64)

    #keras dense layer
    self.dense1 = layers.Dense(64)
    self.dense2 = layers.Dense(num_classes)
    self.relu = MyReLU()

  def call(self, input_tensor):
    # x=tf.nn.relu(self.dense1(input_tensor))
    x = self.relu(self.dense1(input_tensor))
    return self.dense2(x)


In [12]:
model = MyModel()
# model.summary()
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(),
    metrics=["accuracy"],
)

model.fit(x_train, y_train, batch_size=32, epochs=2, verbose=2)
model.evaluate(x_test, y_test, batch_size=32, verbose=2)

Epoch 1/2
1875/1875 - 3s - loss: 0.3104 - accuracy: 0.9138 - 3s/epoch - 2ms/step
Epoch 2/2
1875/1875 - 3s - loss: 0.1506 - accuracy: 0.9568 - 3s/epoch - 2ms/step
313/313 - 0s - loss: 0.1309 - accuracy: 0.9610 - 435ms/epoch - 1ms/step


[0.13089624047279358, 0.9610000252723694]